In [4]:
# import pyodbc
import re

# Define a function to connect to SQL Server and fetch table metadata
def get_table_metadata(server, database, username, password):
    conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Fetch metadata for all tables
    tables_metadata = {}
    cursor.execute("SELECT TABLE_NAME, COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = 'dbo'")
    for row in cursor.fetchall():
        table_name = row.TABLE_NAME
        column_name = row.COLUMN_NAME
        data_type = row.DATA_TYPE
        if table_name not in tables_metadata:
            tables_metadata[table_name] = []
        tables_metadata[table_name].append({'name': column_name, 'type': data_type})

    conn.close()
    return tables_metadata

# Function to identify specific columns
def identify_columns(metadata):
    identification_results = {}
    for table, columns in metadata.items():
        identification_results[table] = {}
        for column in columns:
            column_name = column['name']
            column_type = column['type']
            # Use regular expressions to identify specific columns
            if re.search(r'\b(?:name)\b', column_name, re.IGNORECASE):
                identification_results[table][column_name] = 'Name'
            elif re.search(r'\b(?:address)\b', column_name, re.IGNORECASE):
                identification_results[table][column_name] = 'Address'
            elif re.search(r'\b(?:dob|birthdate|dateofbirth)\b', column_name, re.IGNORECASE):
                identification_results[table][column_name] = 'DOB'
            elif re.search(r'\b(?:phone|mobile|contact)\b', column_name, re.IGNORECASE):
                identification_results[table][column_name] = 'Phone Number'

    return identification_results

def identify_columns_Count(metadata):
    identification_results_count = {'Name': 0, 'Address': 0, 'DOB': 0, 'Phone Number': 0}
    for table, columns in metadata.items():
        for column in columns:
            column_name = column['name']
            column_type = column['type']
            # Use regular expressions to identify specific columns
            if re.search(r'\b(?:name)\b', column_name, re.IGNORECASE):
                identification_results_count['Name'] += 1
            elif re.search(r'\b(?:address)\b', column_name, re.IGNORECASE):
                identification_results_count['Address'] += 1
            elif re.search(r'\b(?:dob|birthdate|dateofbirth)\b', column_name, re.IGNORECASE):
                identification_results_count['DOB'] += 1
            elif re.search(r'\b(?:phone|mobile|contact)\b', column_name, re.IGNORECASE):
                identification_results_count['Phone Number'] += 1

    return identification_results_count

def get_count_metadata(server, database, username, password):
    conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Fetch metadata for all tables
    tables_metadata = {}
    cursor.execute("SELECT TABLE_NAME, COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = 'dbo'")
    for row in cursor.fetchall():
        table_name = row.TABLE_NAME
        column_name = row.COLUMN_NAME
        data_type = row.DATA_TYPE
        if table_name not in tables_metadata:
            tables_metadata[table_name] = []
        tables_metadata[table_name].append({'name': column_name, 'type': data_type})

    conn.close()
    return tables_metadata

# Example usage
server = 'dmoncloud-sqlserver.database.windows.net'
database = 'SRC'
username = 'nishchay'
password = 'Welcome@2o23!'

metadata = get_table_metadata(server, database, username, password)
identification_results = identify_columns(metadata)

metadata_Count= get_count_metadata(server, database, username, password)
identification_results_count = identify_columns_Count(metadata_Count)

# Print the identification results
for table, columns in identification_results.items():
    print(f'\nTable: {table}')
    for column, identifier in columns.items():
        print(f'Column: {column} -> {identifier}')
        
print("\nSummary of columns:")
for identifier, count in identification_results_count.items():
    print(f'{identifier}: {count}')



Table: N_ProductSubcategory

Table: Updated_phone_number

Table: Nis_ODBC2

Table: InsuranceData

Table: Department_group

Table: stg_insuranceData

Table: DW_Employee_SRC
Column: BirthDate -> DOB
Column: Phone -> Phone Number

Table: stage_insuranceData_SCD2

Table: CustomerOrders_YCOPY
Column: NAME -> Name

Table: SuppliesData

Table: Row_Count_Monitor

Table: Datacustomer
Column: BirthDate -> DOB
Column: Phone -> Phone Number

Table: TempCustomerOrders_Y
Column: NAME -> Name

Table: check_data
Column: BirthDate -> DOB
Column: Phone -> Phone Number

Table: DifferingRows
Column: NAME -> Name

Table: N_Product

Table: QUARTERLY_SALES

Table: EMP_SALES_QTR

Table: TOTALAMOUNT

Table: WEEKLY_SALES_BY_STORE

Table: Y_IndianHolidays

Table: Y_Table_Count

Table: Nandini_Insurance_Data

Table: tablesrowcount_N

Table: Nandu

Table: Nanditha_EMP
Column: NAME -> Name

Table: check_customer
Column: BirthDate -> DOB
Column: Phone -> Phone Number

Table: anu_table
Column: name -> Name
Column: a

In [3]:
import pyodbc
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Define a function to connect to SQL Server and fetch table metadata
def get_table_metadata(server, database, username, password):
    conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Fetch metadata for all tables
    tables_metadata = {}
    cursor.execute("SELECT TABLE_NAME, COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = 'dbo'")
    for row in cursor.fetchall():
        table_name = row.TABLE_NAME
        column_name = row.COLUMN_NAME
        data_type = row.DATA_TYPE
        if table_name not in tables_metadata:
            tables_metadata[table_name] = []
        tables_metadata[table_name].append({'name': column_name, 'type': data_type})

    conn.close()
    return tables_metadata

# Function to identify specific columns with enhanced features
def identify_columns(metadata, custom_rules=None):
    identification_results = {}
    for table, columns in metadata.items():
        identification_results[table] = {}
        for column in columns:
            column_name = column['name']
            column_type = column['type']
            # Use regular expressions to identify specific columns
            identifier = identify_column_name(column_name, custom_rules)
            if identifier:
                identification_results[table][column_name] = identifier

    return identification_results

# Function to identify column name based on custom rules and fuzzy matching
def identify_column_name(column_name, custom_rules=None):
    if custom_rules:
        # Apply custom rules for column identification
        for identifier, patterns in custom_rules.items():
            for pattern in patterns:
                if re.search(pattern, column_name, re.IGNORECASE):
                    return identifier
    
    # Fuzzy matching for additional flexibility
    matches = process.extract(column_name.lower(), ["name", "address", "dob", "phone number"], scorer=fuzz.token_sort_ratio)
    best_match = max(matches, key=lambda x: x[1])
    if best_match[1] > 80:  # Adjust threshold as needed
        return best_match[0].title().replace(" ", "")
    
    return None

# Example usage
server = 'dmoncloud-sqlserver.database.windows.net'
database = 'SRC'
username = 'nishchay'
password = 'Welcome@2o23!'

# Custom rules for column identification (optional)
custom_rules = {
    'Name': ['name'],
    'Address': ['address'],
    'DOB': ['dob', 'birthdate', 'dateofbirth'],
    'Phone Number': ['phone', 'mobile', 'contact']
}

metadata = get_table_metadata(server, database, username, password)
identification_results = identify_columns(metadata, custom_rules)

# Print the identification results
for table, columns in identification_results.items():
    print(f'\nTable: {table}')
    for column, identifier in columns.items():
        print(f'Column: {column} -> {identifier}')


C:\Users\Nischay\songs\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')



Table: N_ProductSubcategory
Column: EnglishProductName -> Name
Column: SpanishProductName -> Name
Column: FrenchProductName -> Name

Table: Updated_phone_number
Column: Phone_Number -> Phone Number
Column: TransformedPhoneNumbe -> Phone Number

Table: Nis_ODBC2

Table: InsuranceData

Table: Department_group
Column: Departmentgroupname -> Name

Table: stg_insuranceData

Table: DW_Employee_SRC
Column: FirstName -> Name
Column: LastName -> Name
Column: MiddleName -> Name
Column: NameStyle -> Name
Column: BirthDate -> DOB
Column: EmailAddress -> Address
Column: Phone -> Phone Number
Column: EmergencyContactName -> Name
Column: EmergencyContactPhone -> Phone Number
Column: DepartmentName -> Name

Table: stage_insuranceData_SCD2

Table: CustomerOrders_YCOPY
Column: NAME -> Name

Table: SuppliesData

Table: Row_Count_Monitor
Column: Table_Name -> Name

Table: Datacustomer
Column: FirstName -> Name
Column: MiddleName -> Name
Column: LastName -> Name
Column: NameStyle -> Name
Column: BirthDate

In [5]:
import pyodbc
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Define a function to connect to SQL Server and fetch table metadata
def get_table_metadata(server, database, username, password):
    conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Fetch metadata for all tables
    tables_metadata = {}
    cursor.execute("SELECT TABLE_NAME, COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = 'dbo'")
    for row in cursor.fetchall():
        table_name = row.TABLE_NAME
        column_name = row.COLUMN_NAME
        data_type = row.DATA_TYPE
        if table_name not in tables_metadata:
            tables_metadata[table_name] = []
        tables_metadata[table_name].append({'name': column_name, 'type': data_type})

    conn.close()
    return tables_metadata

# Function to identify specific columns with enhanced features
def identify_columns(metadata, custom_rules=None):
    identification_results = {}
    for table, columns in metadata.items():
        identification_results[table] = {}
        for column in columns:
            column_name = column['name']
            column_type = column['type']
            # Use regular expressions to identify specific columns
            identifier = identify_column_name(column_name, custom_rules)
            if identifier:
                identification_results[table][column_name] = identifier

    return identification_results

# Function to identify column name based on custom rules and fuzzy matching
def identify_column_name(column_name, custom_rules=None):
    if custom_rules:
        # Apply custom rules for column identification
        for identifier, patterns in custom_rules.items():
            for pattern in patterns:
                if re.search(pattern, column_name, re.IGNORECASE):
                    return identifier
    
    # Fuzzy matching for additional flexibility
    matches = process.extract(column_name.lower(), ["name", "address", "dob", "phone number"], scorer=fuzz.token_sort_ratio)
    best_match = max(matches, key=lambda x: x[1])
    if best_match[1] > 80:  # Adjust threshold as needed
        return best_match[0].title().replace(" ", "")
    
    return None

# Function to count identified columns for each category
def identify_columns_count(metadata):
    identification_results_count = {'Name': 0, 'Address': 0, 'DOB': 0, 'Phone Number': 0}
    for table, columns in metadata.items():
        for column in columns:
            column_name = column['name']
            column_type = column['type']
            # Use regular expressions to identify specific columns
            if re.search(r'\b(?:name)\b', column_name, re.IGNORECASE):
                identification_results_count['Name'] += 1
            elif re.search(r'\b(?:address)\b', column_name, re.IGNORECASE):
                identification_results_count['Address'] += 1
            elif re.search(r'\b(?:dob|birthdate|dateofbirth)\b', column_name, re.IGNORECASE):
                identification_results_count['DOB'] += 1
            elif re.search(r'\b(?:phone|mobile|contact)\b', column_name, re.IGNORECASE):
                identification_results_count['Phone Number'] += 1

    return identification_results_count

# Function to get metadata with count for each column category
def get_metadata_with_count(server, database, username, password):
    metadata = get_table_metadata(server, database, username, password)
    identification_results_count = identify_columns_count(metadata)
    return metadata, identification_results_count

# Example usage
server = 'dmoncloud-sqlserver.database.windows.net'
database = 'SRC'
username = 'nishchay'
password = 'Welcome@2o23!'

# Custom rules for column identification (optional)
custom_rules = {
    'Name': ['name'],
    'Address': ['address'],
    'DOB': ['dob', 'birthdate', 'dateofbirth'],
    'Phone Number': ['phone', 'mobile', 'contact']
}

metadata, identification_results_count = get_metadata_with_count(server, database, username, password)
identification_results = identify_columns(metadata, custom_rules)

# Print the identification results
for table, columns in identification_results.items():
    print(f'\nTable: {table}')
    for column, identifier in columns.items():
        print(f'Column: {column} -> {identifier}')
        
print("\nSummary of columns:")
for identifier, count in identification_results_count.items():
    print(f'{identifier}: {count}')



Table: N_ProductSubcategory
Column: EnglishProductName -> Name
Column: SpanishProductName -> Name
Column: FrenchProductName -> Name

Table: Updated_phone_number
Column: Phone_Number -> Phone Number
Column: TransformedPhoneNumbe -> Phone Number

Table: Nis_ODBC2

Table: InsuranceData

Table: Department_group
Column: Departmentgroupname -> Name

Table: stg_insuranceData

Table: DW_Employee_SRC
Column: FirstName -> Name
Column: LastName -> Name
Column: MiddleName -> Name
Column: NameStyle -> Name
Column: BirthDate -> DOB
Column: EmailAddress -> Address
Column: Phone -> Phone Number
Column: EmergencyContactName -> Name
Column: EmergencyContactPhone -> Phone Number
Column: DepartmentName -> Name

Table: stage_insuranceData_SCD2

Table: CustomerOrders_YCOPY
Column: NAME -> Name

Table: SuppliesData

Table: Row_Count_Monitor
Column: Table_Name -> Name

Table: Datacustomer
Column: FirstName -> Name
Column: MiddleName -> Name
Column: LastName -> Name
Column: NameStyle -> Name
Column: BirthDate

In [3]:
import pyodbc
import re

# Define a function to connect to SQL Server and fetch table metadata
def get_table_metadata(server, database, username, password):
    conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Fetch metadata for all tables
    tables_metadata = {}
    cursor.execute("SELECT TABLE_NAME, COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = 'dbo'")
    for row in cursor.fetchall():
        table_name = row.TABLE_NAME
        column_name = row.COLUMN_NAME
        data_type = row.DATA_TYPE
        if table_name not in tables_metadata:
            tables_metadata[table_name] = []
        tables_metadata[table_name].append({'name': column_name, 'type': data_type})

    conn.close()
    return tables_metadata

# Function to identify specific columns based on regular expressions in column data
def identify_columns(metadata):
    sensitive_patterns = {
        r'\b\d{10}\b': 'Phone Number',
        r'\b\d{1,2}/\d{1,2}/\d{4}\b': 'DOB',  # Pattern for Date of Birth
        r'[A-Za-z\s]+': 'Name',  # Pattern for Name (alphabets and spaces)
        r'\d+\s+[\w\s]+,\s+[\w\s]+\s+\d{5}': 'Address'  # Pattern for Address
    }

    identification_results = {}
    for table, columns in metadata.items():
        identification_results[table] = {}
        for column in columns:
            column_name = column['name']
            identification_results[table][column_name] = {}  # Initialize dictionary for column name
            identification_results[table][column_name]['identifier'] = None  # Initialize identifier as None

    # Establish a new connection for fetching column data
    conn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}')
    cursor = conn.cursor()

    for table, columns in metadata.items():
        for column in columns:
            column_name = column['name']
            cursor.execute(f"SELECT [{column_name}] FROM [{table}]")
            rows = cursor.fetchall()
            for row in rows:
                data = row[0]
                for pattern, identifier in sensitive_patterns.items():
                    # Check if any data matches the sensitive pattern
                    if re.search(pattern, str(data), re.IGNORECASE):
                        identification_results[table][column_name]['identifier'] = identifier
                        break  # Move to the next column if a match is found

    conn.close()
    return identification_results

# Example usage
server = 'dmoncloud-sqlserver.database.windows.net'
database = 'SRC'
username = 'nishchay'
password = 'Welcome@2o23!'

metadata = get_table_metadata(server, database, username, password)

identification_results = identify_columns(metadata)

# Print the identification results
for table, columns in identification_results.items():
    print(f'\nTable: {table}')
    for column, info in columns.items():
        identifier = info['identifier']
        print(f'Column: {column} -> Identifier: {identifier}')



Table: N_ProductSubcategory
Column: ProductSubcategoryKey -> Identifier: None
Column: EnglishProductName -> Identifier: None
Column: SpanishProductName -> Identifier: None
Column: FrenchProductName -> Identifier: None

Table: Nis_ODBC2
Column: OrderID -> Identifier: None
Column: OrderDate -> Identifier: None
Column: Region -> Identifier: Name
Column: Rep -> Identifier: Name
Column: Item -> Identifier: Name
Column: Units -> Identifier: None
Column: Unit_Cost -> Identifier: None

Table: InsuranceData
Column: Policy -> Identifier: None
Column: Expiry -> Identifier: None
Column: Location -> Identifier: Name
Column: State -> Identifier: Name
Column: Region -> Identifier: Name
Column: InsuredValue -> Identifier: None
Column: Construction -> Identifier: Name
Column: BusinessType -> Identifier: Name
Column: Earthquake -> Identifier: Name
Column: Flood -> Identifier: Name
Column: Load_date -> Identifier: None

Table: Department_group
Column: DepartmentGroupKey -> Identifier: None
Column: Paren

In [4]:
import pyodbc
import re

# Define a function to connect to SQL Server and fetch table metadata
def get_table_metadata(server, database, username, password):
    conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Fetch metadata for all tables
    tables_metadata = {}
    cursor.execute("SELECT TABLE_NAME, COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = 'dbo'")
    for row in cursor.fetchall():
        table_name = row.TABLE_NAME
        column_name = row.COLUMN_NAME
        data_type = row.DATA_TYPE
        if table_name not in tables_metadata:
            tables_metadata[table_name] = []
        tables_metadata[table_name].append({'name': column_name, 'type': data_type})

    conn.close()
    return tables_metadata

# Function to identify specific columns based on regular expressions in column data
def identify_columns(metadata):
    sensitive_patterns = {
        r'\d{10}': 'Phone Number',
        r'\b\d{1,2}/\d{1,2}/\d{4}\b': 'DOB',  # Pattern for Date of Birth
        r'[A-Za-z\s]+': 'Name',  # Pattern for Name (alphabets and spaces)
        r'\d+\s+[\w\s]+,\s+\w+\s+\d+': 'Address'  # Pattern for Address
    }

    identification_results = {}
    for table, columns in metadata.items():
        identification_results[table] = {}
        for column in columns:
            column_name = column['name']
            # Initialize identification as 'Other' by default
            #identification_results[table][column_name] = 'Other'
        
        # Fetch all column data for the table
        cursor.execute(f"SELECT * FROM [{table}]")
        rows = cursor.fetchall()
        
        for row in rows:
            for column_info, data in zip(cursor.description, row):
                column_name = column_info[0]  # Accessing the first element of the tuple, which should be the column name
                for pattern, identifier in sensitive_patterns.items():
                    # Check if the column name matches a pattern
                    if re.match(r'(address|dob|phone number|name)', column_name, re.IGNORECASE):
                        # Check if any data matches the sensitive pattern
                        if re.search(pattern, str(data), re.IGNORECASE):
                            identification_results[table][column_name] = identifier
                            break  # Move to the next column if a match is found

    return identification_results

# Example usage
server = 'dmoncloud-sqlserver.database.windows.net'
database = 'SRC'
username = 'nishchay'
password = 'Welcome@2o23!'

metadata = get_table_metadata(server, database, username, password)

# Establish a new connection for fetching column data
conn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}')
cursor = conn.cursor()

identification_results = identify_columns(metadata)

# Print the identification results
for table, columns in identification_results.items():
    print(f'\nTable: {table}')
    for column, identifier in columns.items():
        print(f'Column: {column} -> {identifier}')

# Close the connection
conn.close()



Table: N_ProductSubcategory

Table: Nis_ODBC2

Table: InsuranceData

Table: Department_group

Table: stg_insuranceData

Table: DW_Employee_SRC
Column: NameStyle -> Name

Table: stage_insuranceData_SCD2

Table: CustomerOrders_YCOPY
Column: NAME -> Name

Table: SuppliesData

Table: Row_Count_Monitor

Table: Datacustomer
Column: AddressLine1 -> Name
Column: AddressLine2 -> Name

Table: TempCustomerOrders_Y
Column: NAME -> Name

Table: check_data
Column: AddressLine1 -> Name
Column: AddressLine2 -> Name

Table: DifferingRows
Column: NAME -> Name

Table: N_Product

Table: QUARTERLY_SALES

Table: EMP_SALES_QTR

Table: TOTALAMOUNT

Table: WEEKLY_SALES_BY_STORE

Table: Y_IndianHolidays

Table: Y_Table_Count

Table: Nandini_Insurance_Data

Table: tablesrowcount_N

Table: Nandu

Table: Nanditha_EMP
Column: NAME -> Name

Table: anu_table
Column: name -> Name
Column: address -> Name

Table: Insurance_Data

Table: Employee_count_history

Table: Dl_lnd_Customer
Column: AddressLine1 -> Name
Column: 

In [4]:
import pyodbc
import re

# Define a function to connect to SQL Server and fetch table metadata
def get_table_metadata(server, database, username, password):
    conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Fetch metadata for all tables
    tables_metadata = {}
    cursor.execute("SELECT TABLE_NAME, COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = 'dbo'")
    for row in cursor.fetchall():
        table_name = row.TABLE_NAME
        column_name = row.COLUMN_NAME
        data_type = row.DATA_TYPE
        if table_name not in tables_metadata:
            tables_metadata[table_name] = []
        tables_metadata[table_name].append({'name': column_name, 'type': data_type})

    conn.close()
    return tables_metadata

# Function to identify specific columns based on regular expressions in column data
def identify_columns(metadata):
    sensitive_patterns = {
        r'\b\d{2}[-/]\d{2}[-/]\d{4}\b': 'Date',
        r'\d{10}': 'Phone Number',
        r'\b\d{1,2}/\d{1,2}/\d{4}\b': 'DOB',  # Pattern for Date of Birth
        r'[A-Za-z\s]+': 'Name'  # Pattern for Name (alphabets and spaces)
    }

    identification_results = {}
    for table, columns in metadata.items():
        identification_results[table] = {}
        for column in columns:
            column_name = column['name']
            identification_results[table][column_name] = {}  # Initialize dictionary for column name
            identification_results[table][column_name]['identifier'] = None  # Initialize identifier as None
        
        # Fetch all column data for the table
        cursor.execute(f"SELECT * FROM [{table}]")
        rows = cursor.fetchall()
        
        for row in rows:
            for column_name, data in zip(cursor.description, row):
                for pattern, identifier in sensitive_patterns.items():
                    # Check if any data matches the sensitive pattern
                    if re.search(pattern, str(data), re.IGNORECASE):
                        identification_results[table][column_name[0]]['identifier'] = identifier
                        break  # Move to the next column if a match is found

    return identification_results

# Example usage
server = 'dmoncloud-sqlserver.database.windows.net'
database = 'SRC'
username = 'nishchay'
password = 'Welcome@2o23!'

metadata = get_table_metadata(server, database, username, password)

# Establish a new connection for fetching column data
conn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}')
cursor = conn.cursor()

identification_results = identify_columns(metadata)

# Count the occurrences of each identifier
total_address_columns = 0
total_dob_columns = 0
total_name_columns = 0
total_phone_number_columns = 0

for table, columns in identification_results.items():
    for column, info in columns.items():
        identifier = info['identifier']
        if identifier == 'Address':
            total_address_columns += 1
        elif identifier == 'DOB':
            total_dob_columns += 1
        elif identifier == 'Name':
            total_name_columns += 1
        elif identifier == 'Phone Number':
            total_phone_number_columns += 1

# Print the summary
print(f"Total Address Columns: {total_address_columns}")
print(f"Total DOB Columns: {total_dob_columns}")
print(f"Total Name Columns: {total_name_columns}")
print(f"Total Phone Number Columns: {total_phone_number_columns}")

# Close the connection
conn.close()


Total Address Columns: 0
Total DOB Columns: 0
Total Name Columns: 419
Total Phone Number Columns: 14


In [4]:
import pyodbc
import re

# Define a function to connect to SQL Server and fetch table metadata
def get_table_metadata(server, database, username, password):
    conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Fetch metadata for all tables
    tables_metadata = {}
    cursor.execute("SELECT TABLE_NAME, COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = 'dbo'")
    for row in cursor.fetchall():
        table_name = row.TABLE_NAME
        column_name = row.COLUMN_NAME
        data_type = row.DATA_TYPE
        if table_name not in tables_metadata:
            tables_metadata[table_name] = []
        tables_metadata[table_name].append({'name': column_name, 'type': data_type})

    conn.close()
    return tables_metadata

# Function to identify specific columns based on regular expressions in column data
def identify_columns(metadata):
    sensitive_patterns = {
        r'\d{10}': 'Phone Number',
        r'\b\d{1,2}/\d{1,2}/\d{4}\b': 'DOB',  # Pattern for Date of Birth
        r'[A-Za-z\s]+': 'Name',  # Pattern for Name (alphabets and spaces)
        r'\d+\s+[\w\s]+,\s+\w+\s+\d+': 'Address'  # Pattern for Address
    }
    sensitive_keywords = {
        'dob': 'DOB',  # Keywords indicating Date of Birth column
        'address': 'Address',  # Keywords indicating Address column
        'phoneNumber': 'Phone Number',
        'name':'Name'
    }

    identification_results = {}
    for table, columns in metadata.items():
        identification_results[table] = {}
        for column in columns:
            column_name = column['name']
            identification_results[table][column_name] = {'identifier': None}  # Initialize identifier as None
        
        # Fetch all column data for the table
        cursor.execute(f"SELECT * FROM [{table}]")
        rows = cursor.fetchall()
        
        for row in rows:
            for column_name, data in zip(cursor.description, row):
                for pattern, identifier in sensitive_patterns.items():
                    # Check if any data matches the sensitive pattern
                    if re.search(pattern, str(data), re.IGNORECASE):
                        identification_results[table][column_name[0]]['identifier'] = identifier
                        break  # Move to the next column if a match is found
                # Check for sensitive keywords in column names
                for keyword, identifier in sensitive_keywords.items():
                    if keyword.lower() in column_name[0].lower():
                        identification_results[table][column_name[0]]['identifier'] = identifier

    return identification_results

# Example usage
server = 'dmoncloud-sqlserver.database.windows.net'
database = 'SRC'
username = 'nishchay'
password = 'Welcome@2o23!'

metadata = get_table_metadata(server, database, username, password)

# Establish a new connection for fetching column data
conn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}')
cursor = conn.cursor()

identification_results = identify_columns(metadata)

# Print the identification results
for table, columns in identification_results.items():
    print(f'\nTable: {table}')
    for column, info in columns.items():
        identifier = info['identifier']
        if identifier:
            print(f'Column: {column} -> Identifier: {identifier}')

# Close the connection
conn.close()



Table: N_ProductSubcategory

Table: Nis_ODBC2

Table: InsuranceData

Table: Department_group
Column: Departmentgroupname -> Identifier: Name

Table: stg_insuranceData

Table: DW_Employee_SRC
Column: FirstName -> Identifier: Name
Column: LastName -> Identifier: Name
Column: MiddleName -> Identifier: Name
Column: NameStyle -> Identifier: Name
Column: EmailAddress -> Identifier: Address
Column: EmergencyContactName -> Identifier: Name
Column: DepartmentName -> Identifier: Name

Table: stage_insuranceData_SCD2

Table: CustomerOrders_YCOPY
Column: NAME -> Identifier: Name

Table: SuppliesData

Table: Row_Count_Monitor

Table: Datacustomer
Column: FirstName -> Identifier: Name
Column: MiddleName -> Identifier: Name
Column: LastName -> Identifier: Name
Column: NameStyle -> Identifier: Name
Column: EmailAddress -> Identifier: Address
Column: AddressLine1 -> Identifier: Address
Column: AddressLine2 -> Identifier: Address

Table: TempCustomerOrders_Y
Column: NAME -> Identifier: Name

Table: che

In [8]:
import pyodbc
import re

# Define a function to connect to SQL Server and fetch table metadata
def get_table_metadata(server, database, username, password):
    conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Fetch metadata for all tables
    tables_metadata = {}
    cursor.execute("SELECT TABLE_NAME, COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = 'dbo'")
    for row in cursor.fetchall():
        table_name = row.TABLE_NAME
        column_name = row.COLUMN_NAME
        data_type = row.DATA_TYPE
        if table_name not in tables_metadata:
            tables_metadata[table_name] = []
        tables_metadata[table_name].append({'name': column_name, 'type': data_type})

    conn.close()
    return tables_metadata

# Function to identify specific columns based on regular expressions in column data
def identify_columns(metadata):
    sensitive_patterns = {
        r'^\(\d{3}\) \d{3}-\d{4}$': 'Phone Number',
        r'\b\d{1,2}/\d{1,2}/\d{4}\b': 'DOB',  # Pattern for Date of Birth
        r'\d+\s+[\w\s]+,\s+\w+\s+\d+': 'Address',  # Pattern for Address
        r"^[A-Za-z]+(?:[ '-][A-Za-z]+)*$":'Name'
    }
#     sensitive_keywords = {
#         'dob': 'DOB',  # Keywords indicating Date of Birth column
#         'address': 'Address',  # Keywords indicating Address column
#         'phoneNumber': 'Phone Number',
#         'name': 'Name'
#     }

    identification_results = {}
    for table, columns in metadata.items():
        identification_results[table] = {}
        for column in columns:
            column_name = column['name']
            identification_results[table][column_name] = {'identifier': None}  # Initialize identifier as None
        
        # Fetch all column data for the table
        cursor.execute(f"SELECT * FROM [{table}]")
        rows = cursor.fetchall()
        
        for row in rows:
            for column_name, data in zip(cursor.description, row):
                for pattern, identifier in sensitive_patterns.items():
                    # Check if any data matches the sensitive pattern
                    if re.search(pattern, str(data), re.IGNORECASE):
                        identification_results[table][column_name[0]]['identifier'] = identifier
                        break  # Move to the next column if a match is found
                # Check for sensitive keywords in column names
                for keyword, identifier in sensitive_keywords.items():
                    if keyword.lower() in column_name[0].lower():
                        identification_results[table][column_name[0]]['identifier'] = identifier

    return identification_results

# Example usage
server = 'dmoncloud-sqlserver.database.windows.net'
database = 'SRC'
username = 'nishchay'
password = 'Welcome@2o23!'

metadata = get_table_metadata(server, database, username, password)

# Establish a new connection for fetching column data
conn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}')
cursor = conn.cursor()

identification_results = identify_columns(metadata)

# Collect identified columns counts
total_address_columns = 0
total_dob_columns = 0
total_name_columns = 0
total_phone_number_columns = 0

for table, columns in identification_results.items():
    for column, info in columns.items():
        identifier = info['identifier']
        if identifier == 'Address':
            total_address_columns += 1
        elif identifier == 'DOB':
            total_dob_columns += 1
        elif identifier == 'Name':
            total_name_columns += 1
        elif identifier == 'Phone Number':
            total_phone_number_columns += 1

# Print the summary
print(f"Total Address Columns: {total_address_columns}")
print(f"Total DOB Columns: {total_dob_columns}")
print(f"Total Name Columns: {total_name_columns}")
print(f"Total Phone Number Columns: {total_phone_number_columns}")

# Close the connection
conn.close()


NameError: name 'sensitive_keywords' is not defined

In [9]:
import pyodbc
import re

# Define a function to connect to SQL Server and fetch table metadata
def get_table_metadata(server, database, username, password):
    conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Fetch metadata for all tables
    tables_metadata = {}
    cursor.execute("SELECT TABLE_NAME, COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = 'dbo'")
    for row in cursor.fetchall():
        table_name = row.TABLE_NAME
        column_name = row.COLUMN_NAME
        data_type = row.DATA_TYPE
        if table_name not in tables_metadata:
            tables_metadata[table_name] = []
        tables_metadata[table_name].append({'name': column_name, 'type': data_type})

    conn.close()
    return tables_metadata

# Function to identify specific columns based on regular expressions in column data
def identify_columns(metadata):
    sensitive_patterns = {
        r'^\(\d{3}\) \d{3}-\d{4}$': 'Phone Number',
        r'\b\d{1,2}/\d{1,2}/\d{4}\b': 'DOB',  # Pattern for Date of Birth
        r'\d+\s+[\w\s]+,\s+\w+\s+\d+': 'Address',  # Pattern for Address
        r'[A-Za-z\s]+': 'Name',  # Pattern for Name (alphabets and spaces)
        r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b': 'Email'  # Pattern for Email

    }

    identification_results = {}
    for table, columns in metadata.items():
        identification_results[table] = {}
        for column in columns:
            column_name = column['name']
            identification_results[table][column_name] = {'identifier': None}  # Initialize identifier as None

        # Fetch all column data for the table
        cursor.execute(f"SELECT * FROM [{table}]")
        rows = cursor.fetchall()

        for row in rows:
            for column_name, data in zip(cursor.description, row):
                for pattern, identifier in sensitive_patterns.items():
                    # Check if any data matches the sensitive pattern
                    if re.search(pattern, str(data), re.IGNORECASE):
                        identification_results[table][column_name[0]]['identifier'] = identifier
                        break  # Move to the next column if a match is found

    return identification_results

# Example usage
server = 'dmoncloud-sqlserver.database.windows.net'
database = 'SRC'
username = 'nishchay'
password = 'Welcome@2o23!'

metadata = get_table_metadata(server, database, username, password)

# Establish a new connection for fetching column data
conn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}')
cursor = conn.cursor()

identification_results = identify_columns(metadata)

# Print the identification results
for table, columns in identification_results.items():
    print(f'\nTable: {table}')
    for column, info in columns.items():
        identifier = info['identifier']
        if identifier:
            print(f'Column: {column} -> Identifier: {identifier}')

# Close the connection
conn.close()



Table: N_ProductSubcategory

Table: Nis_ODBC2
Column: Region -> Identifier: Name
Column: Rep -> Identifier: Name
Column: Item -> Identifier: Name

Table: InsuranceData
Column: Location -> Identifier: Name
Column: State -> Identifier: Name
Column: Region -> Identifier: Name
Column: Construction -> Identifier: Name
Column: BusinessType -> Identifier: Name
Column: Earthquake -> Identifier: Name
Column: Flood -> Identifier: Name

Table: Department_group
Column: Departmentgroupname -> Identifier: Name

Table: stg_insuranceData
Column: Location -> Identifier: Name
Column: State -> Identifier: Name
Column: Region -> Identifier: Name
Column: Construction -> Identifier: Name
Column: BusinessType -> Identifier: Name
Column: Earthquake -> Identifier: Name
Column: Flood -> Identifier: Name
Column: Active_status -> Identifier: Name

Table: DW_Employee_SRC
Column: ParentEmployeeKey -> Identifier: Name
Column: ParentEmployeeNationalIDAlternateKey -> Identifier: Name
Column: FirstName -> Identifier: 

In [24]:
import pyodbc
import re

# Define a function to connect to SQL Server and fetch table metadata
def get_table_metadata(server, database, username, password):
    conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Fetch metadata for all tables
    tables_metadata = {}
    cursor.execute("SELECT TABLE_NAME, COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = 'dbo'")
    for row in cursor.fetchall():
        table_name = row.TABLE_NAME
        column_name = row.COLUMN_NAME
        data_type = row.DATA_TYPE
        if table_name not in tables_metadata:
            tables_metadata[table_name] = []
        tables_metadata[table_name].append({'name': column_name, 'type': data_type})

    conn.close()
    return tables_metadata

# Function to identify specific columns based on regular expressions in column data
def identify_columns(metadata):
    sensitive_patterns = {
        r'\d{2,4}-\d{2,4}-\d{2,4}': 'Date',  # Pattern for Date
        r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b': 'Email',  # Pattern for Email
        r'(?<!\d)\b(?:\+\d{1,3}\s?)?(?:\d{1,4}[\s.-]?)?\d{9,12}[\s.-]?\d{9,12}[\s.-]?\d{9,12}\b(?!\d)': 'Phone Number',  # Pattern for Phone Number
        r'\d+\s+\w+.*?,\s*\w+.*?,\s*\w+\s+\d+': 'Address',  # Pattern for Address
        r"[A-Za-z]+(?: [A-Za-z]+)+": 'Name',  # Pattern for Name
    }

    identification_results = {}
    for table, columns in metadata.items():
        identification_results[table] = {}
        for column in columns:
            column_name = column['name']
            # Initialize identification as 'Other' by default
            #identification_results[table][column_name] = 'Other'
        
        # Fetch all column data for the table
        cursor.execute(f"SELECT * FROM [{table}]")
        #cursor.execute(f"SELECT * FROM test")
        rows = cursor.fetchall()
        
        for row in rows:
            for column_name, data in zip(cursor.description, row):
                for pattern, identifier in sensitive_patterns.items():
                    # Check if any data matches the sensitive pattern
                    if re.search(pattern, str(data)):
                        identification_results[table][column_name[0]] = identifier
                        break  # Move to the next column if a match is found

    return identification_results

# Example usage
server = 'dmoncloud-sqlserver.database.windows.net'
database = 'SRC'
username = 'nishchay'
password = 'Welcome@2o23!'

metadata = get_table_metadata(server, database, username, password)

# Establish a new connection for fetching column data
conn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}')
cursor = conn.cursor()

identification_results = identify_columns(metadata)

print(f'\nTable: {table}')
    
print(f'Column: {column} -> {identifier}')
# Print the identification results
for table, columns in identification_results.items():
    print(f'\nTable: {table}')
    for column, identifier in columns.items():
        print(f'Column: {column} -> {identifier}')

# Close the connection
conn.close()



Table: dw_lnd_suppliesData
Column: Units -> Name

Table: N_ProductSubcategory

Table: Nis_ODBC2
Column: OrderDate -> Date
Column: Item -> Name

Table: InsuranceData
Column: Expiry -> Date
Column: Load_date -> Date
Column: BusinessType -> Name
Column: Construction -> Name

Table: Department_group
Column: Departmentgroupname -> Name

Table: stg_insuranceData
Column: Expiry -> Date
Column: Load_date -> Date
Column: State -> Name
Column: BusinessType -> Name
Column: Construction -> Name

Table: DW_Employee_SRC
Column: HireDate -> Date
Column: BirthDate -> Date
Column: EmailAddress -> Email
Column: Phone -> Date
Column: EmergencyContactPhone -> Date
Column: StartDate -> Date
Column: UpdatedDate -> Date
Column: EndDate -> Date
Column: Title -> Date
Column: LoginID -> Date
Column: SalesPersonFlag -> Date

Table: stage_insuranceData_SCD2
Column: Expiry -> Date
Column: State -> Name
Column: Load_date -> Date
Column: Construction -> Name
Column: BusinessType -> Name

Table: CustomerOrders_YCOPY

In [2]:
import pyodbc
import re

# Define a function to connect to SQL Server and fetch table metadata
def get_table_metadata(server, database, username, password):
    conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Fetch metadata for all tables
    tables_metadata = {}
    cursor.execute("SELECT TABLE_NAME, COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = 'dbo'")
    for row in cursor.fetchall():
        table_name = row.TABLE_NAME
        column_name = row.COLUMN_NAME
        data_type = row.DATA_TYPE
        if table_name not in tables_metadata:
            tables_metadata[table_name] = []
        tables_metadata[table_name].append({'name': column_name, 'type': data_type})

    conn.close()
    return tables_metadata

# Function to identify specific columns based on regular expressions in column data
def identify_columns(metadata):
    sensitive_patterns = {
        r'\d{2,4}[-/]\d{2}[-/]\d{2,4}': 'Date',  # Pattern for Date
        r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b': 'Email',  # Pattern for Email
        r'\+\d{1,3}(?:\s?\d{10})?': 'Phone Number',  # Pattern for Phone Number
        r'^[^\w\s]\D\d{20,}$': 'Address',  # Pattern for Address
        r"[A-Za-z][.,' '](?: [A-Za-z]+)+": 'Name',  # Pattern for Name
    }

    identification_results = {}
    
    for table, columns in metadata.items():
        identification_results[table] = {}
        for column in columns:
            column_name = column['name']
            # Initialize identification as 'Other' by default
            #identification_results[table][column_name] = 'Other'
        
        # Fetch all column data for the table
        cursor.execute(f"SELECT * FROM [{table}]")
        rows = cursor.fetchall()
        
        for row in rows:
            for column_name, data in zip(cursor.description, row):
                for pattern, identifier in sensitive_patterns.items():
                    # Check if any data matches the sensitive pattern
                    if re.search(pattern, str(data)):
                        identification_results[table][column_name[0]] = identifier
                        break  # Move to the next column if a match is found

    return identification_results

# Example usage
server = 'dmoncloud-sqlserver.database.windows.net'
database = 'SRC'
username = 'nishchay'
password = 'Welcome@2o23!'

metadata = get_table_metadata(server, database, username, password)

# Establish a new connection for fetching column data
conn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}')
cursor = conn.cursor()

identification_results = identify_columns(metadata)

# Print the identification results
for table, columns in identification_results.items():
    print(f'\nTable: {table}')
    for column, identifier in columns.items():
        print(f'Column: {column} -> {identifier}')

# Close the connection
conn.close()



Table: N_ProductSubcategory

Table: Nis_ODBC2
Column: OrderDate -> Date

Table: InsuranceData
Column: Expiry -> Date
Column: Load_date -> Date

Table: Department_group

Table: stg_insuranceData
Column: Expiry -> Date
Column: Load_date -> Date

Table: DW_Employee_SRC
Column: HireDate -> Date
Column: BirthDate -> Date
Column: EmailAddress -> Email
Column: StartDate -> Date
Column: UpdatedDate -> Date
Column: EndDate -> Date
Column: Title -> Date
Column: SalesPersonFlag -> Date

Table: stage_insuranceData_SCD2
Column: Expiry -> Date
Column: Load_date -> Date

Table: CustomerOrders_YCOPY

Table: SuppliesData
Column: OrderDate -> Date
Column: Load_Date -> Date

Table: Row_Count_Monitor

Table: Datacustomer
Column: BirthDate -> Date
Column: EmailAddress -> Email
Column: DateFirstPurchase -> Date
Column: AddressLine1 -> Name

Table: TempCustomerOrders_Y

Table: check_data
Column: BirthDate -> Date
Column: EmailAddress -> Email
Column: DateFirstPurchase -> Date

Table: DifferingRows

Table: N